In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
import os
import collections

import torch
import torch.nn as nn
from torch.utils.data.dataloader import DataLoader
from tqdm import tqdm
import h5py
import pandas as pd


In [6]:
from param import args

args.output = r'snap/vqa_multilingual'
#args.config = r'configs/vqa.json'



In [8]:
from pretrain.qa_answer_table import AnswerTable,load_lxmert_qa, load_lxmert_from_sgg_and_lxmert_pretrain, load_lxmert_from_pretrain_noqa
from tasks.vqa_model import VQAModel
from tasks.vqa_data import VQADataset, VQATorchDataset, VQAEvaluator

from utils import load_lxmert_sgg

DataTuple = collections.namedtuple("DataTuple", 'dataset loader evaluator')

self.symbolic_embedding_number: 4003


In [14]:
def get_data_tuple(splits: str, bs:int, shuffle=False, drop_last=False) -> DataTuple:
    dset = VQADataset(splits)
    tset = VQATorchDataset(dset, args)
    evaluator = VQAEvaluator(dset)
    data_loader = DataLoader(
        tset, batch_size=bs,
        shuffle=shuffle, num_workers=args.num_workers,
        drop_last=drop_last, pin_memory=True,
        collate_fn=lambda x: x
    )

    return DataTuple(dataset=dset, loader=data_loader, evaluator=evaluator)


class VQA:
    def __init__(self):
        # Datasets
        self.train_tuple = get_data_tuple(
            args.train, bs=args.batch_size, shuffle=True, drop_last=True
        )
        if args.valid != "":
            valid_bsize = args.get("valid_batch_size", 16)
            self.valid_tuple = get_data_tuple(
                args.valid, bs=valid_bsize,
                shuffle=False, drop_last=False
            )
        else:
            self.valid_tuple = None
        
        # Model
        self.model = VQAModel(self.train_tuple.dataset.num_answers)

        # Load pre-trained weights
        if args.load_lxmert is not None:
            print('1')
            self.model.lxrt_encoder.load(args.load_lxmert)
        if args.get("load_lxmert_pretrain", None) is not None:
            print('2')
            load_lxmert_from_pretrain_noqa(args.load_lxmert_pretrain, self.model)
        if args.load_lxmert_qa is not None:
            print('3')
            load_lxmert_qa(args.load_lxmert_qa, self.model,
                           label2ans=self.train_tuple.dataset.label2ans)
        
        # GPU options
        self.model = self.model.cuda()
        if args.multiGPU:
            self.model.lxrt_encoder.multi_gpu()
            self.model.multi_gpu()

        # Loss and Optimizer
        self.bce_loss = nn.BCEWithLogitsLoss()
        if 'bert' in args.optim:
            batch_per_epoch = len(self.train_tuple.loader)
            t_total = int(batch_per_epoch * args.epochs)
            print("BertAdam Total Iters: %d" % t_total)
            from lxrt.optimization import BertAdam
            self.optim = BertAdam(list(self.model.parameters()),
                                  lr=args.lr,
                                  warmup=0.1,
                                  t_total=t_total)
        else:
            self.optim = args.optimizer(self.model.parameters(), args.lr)
        
        # Output Directory
        self.output = args.output
        os.makedirs(self.output, exist_ok=True)

    def train(self, train_tuple, eval_tuple):
        dset, loader, evaluator = train_tuple
        iter_wrapper = (lambda x: tqdm(x, total=len(loader))) if args.tqdm else (lambda x: x)

        best_valid = 0.
        train_results = []
        report_every = args.get("report_every", 100)
        for epoch in range(args.epochs):
            quesid2ans = {}
            for i, batch in iter_wrapper(enumerate(loader)):
                # ques_id, feats, boxes, sent, tags, target = zip(*batch)
                ques_id, feats, boxes, sent, tags, target , obj_labels, obj_confs, attr_labels, attr_confs,img_id,ques_type,multi_ans = zip(*batch)

                self.model.train()
                self.optim.zero_grad()

                target = torch.stack(target).cuda()
                logit = self.model(feats, boxes, sent, tags)
                assert logit.dim() == target.dim() == 2
                loss = self.bce_loss(logit, target)
                loss = loss * logit.size(1)

                loss.backward()
                nn.utils.clip_grad_norm_(self.model.parameters(), 5.)
                self.optim.step()
                train_results.append(pd.Series({"loss":loss.detach().mean().item()}))

                score, label = logit.max(1)
                for qid, l in zip(ques_id, label.cpu().numpy()):
                    ans = dset.label2ans[l]
                    quesid2ans[qid] = ans
                
                if i % report_every == 0 and i > 0:
                    print("Epoch: {}, Iter: {}/{}".format(epoch, i, len(loader)))
                    print("    {}\n~~~~~~~~~~~~~~~~~~\n".format(pd.DataFrame(train_results[-report_every:]).mean()))

            log_str = "\nEpoch %d: Train %0.2f\n" % (epoch, evaluator.evaluate(quesid2ans) * 100.)

            if self.valid_tuple is not None:  # Do Validation
                valid_score = self.evaluate(eval_tuple)
                if valid_score > best_valid and not args.get("special_test", False):
                    best_valid = valid_score
                    self.save("BEST")

                log_str += "Epoch %d: Valid %0.2f\n" % (epoch, valid_score * 100.) + \
                           "Epoch %d: Best %0.2f\n" % (epoch, best_valid * 100.)
            #if epoch >= 5:
            self.save("Epoch{}".format(epoch))
            print(log_str, end='')
            print(args.output)


        self.save("LAST")

    def predict(self, eval_tuple: DataTuple, dump=None):
        """
        Predict the answers to questions in a data split.

        :param eval_tuple: The data tuple to be evaluated.
        :param dump: The path of saved file to dump results.
        :return: A dict of question_id to answer.
        """
        self.model.eval()
        dset, loader, evaluator = eval_tuple
        quesid2ans = {}
        for i, batch in enumerate(tqdm(loader)):
            _ = list(zip(*batch))
            ques_id, feats, boxes, sent, tags = _[:5]#, target = zip(*batch)
            with torch.no_grad():
                #target = torch.stack(target).cuda()
                logit = self.model(feats, boxes, sent, tags)
                score, label = logit.max(1)
                for qid, l in zip(ques_id, label.cpu().numpy()):
                    ans = dset.label2ans[l]
                    quesid2ans[qid] = ans
        if dump is not None:
            evaluator.dump_result(quesid2ans, dump)
        return quesid2ans

    def evaluate(self, eval_tuple: DataTuple, dump=None):
        """Evaluate all data in data_tuple."""
        quesid2ans = self.predict(eval_tuple, dump)
        return eval_tuple.evaluator.evaluate(quesid2ans)

    @staticmethod
    def oracle_score(data_tuple):
        dset, loader, evaluator = data_tuple
        quesid2ans = {}
        for i, (ques_id, feats, boxes, sent, target) in enumerate(loader):
            _, label = target.max(1)
            for qid, l in zip(ques_id, label.cpu().numpy()):
                ans = dset.label2ans[l]
                quesid2ans[qid.item()] = ans
        return evaluator.evaluate(quesid2ans)

    def save(self, name):
        if args.get('german_train', True):
            torch.save(self.model.state_dict(),
                       os.path.join(self.output, "%sGE.pth" % name))   
        elif args.get('german_only', True):
            torch.save(self.model.state_dict(),
                       os.path.join(self.output, "%sGEOnly.pth" % name))   
        else:
            torch.save(self.model.state_dict(),
                       os.path.join(self.output, "%sENonly.pth" % name))

    def load(self, path):
        print("Load model from %s" % path)
        state_dict = torch.load("%s.pth" % path)
        self.model.load_state_dict(state_dict)
        
import os
os.environ['http_proxy'] = "http://135.245.48.34:8000" 
os.environ['https_proxy'] = "http://135.245.48.34:8000" 

# Train (fine-tune) and Save

In [16]:
args.load_lxmert_pretrain = "snap/pretrained/BEST_EVAL_LOSS_enge_LXRT.pth" #"snap/pretrained/CC_Unsupervised_LXRT.pth"


In [17]:
vqa = VQA()

Load 443757 data from split(s) train.
['train']
'train2014_obj36.h5_boxes': returning memmapped view.
'train2014_obj36.h5_objects_id': returning memmapped view.
'train2014_obj36.h5_objects_conf': returning memmapped view.
'train2014_obj36.h5_attrs_id': returning memmapped view.
'train2014_obj36.h5_attrs_conf': returning memmapped view.
data/mscoco_imgfeat/train2014_obj36.h5 82783
Created ['train']


INFO:lxrt.tokenization:loading vocabulary file C:/Users/lab/work/Anna/nlp-tutorial/5-2.BERT/bert-base-multilingual-uncased\vocab.txt


Use 443757 data in torch dataset

data/vocabs/attributes_vocab.txt
data/vocabs/attributes_de_vocab.txt
data/vocabs/objects_vocab.txt
data/vocabs/objects_de_vocab.txt
Load 25994 data from split(s) minival.
['minival']
'val2014_obj36.h5_boxes': returning memmapped view.
'val2014_obj36.h5_objects_id': returning memmapped view.
'val2014_obj36.h5_objects_conf': returning memmapped view.
'val2014_obj36.h5_attrs_id': returning memmapped view.
'val2014_obj36.h5_attrs_conf': returning memmapped view.


INFO:lxrt.tokenization:loading vocabulary file C:/Users/lab/work/Anna/nlp-tutorial/5-2.BERT/bert-base-multilingual-uncased\vocab.txt
INFO:lxrt.tokenization:loading vocabulary file C:/Users/lab/work/Anna/nlp-tutorial/5-2.BERT/bert-base-multilingual-uncased\vocab.txt


data/mscoco_imgfeat/val2014_obj36.h5 40504
Created ['minival']
Use 25994 data in torch dataset

data/vocabs/attributes_vocab.txt
data/vocabs/attributes_de_vocab.txt
data/vocabs/objects_vocab.txt
data/vocabs/objects_de_vocab.txt


INFO:lxrt.modeling:loading archive file C:/Users/lab/work/Anna/nlp-tutorial/5-2.BERT/bert-base-multilingual-uncased
INFO:lxrt.modeling:Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "symbolic_embedding_number": 4003,
  "type_vocab_size": 2,
  "vocab_size": 105879
}



LXRT encoder with 12 l_layers, 0 x_layers, and 0 r_layers.


!! Has 12 layers


INFO:lxrt.modeling:Weights of LXRTFeatureExtraction not initialized from pretrained model: ['bert.embeddings.symbolic_embedding.weight', 'bert.embeddings.token_type_embeddings_visual.weight', 'bert.embeddings.position_embeddings_visual.weight', 'bert.embeddings.visn_fc.weight', 'bert.embeddings.visn_fc.bias', 'bert.embeddings.visn_layer_norm.weight', 'bert.embeddings.visn_layer_norm.bias', 'bert.embeddings.box_fc.weight', 'bert.embeddings.box_fc.bias', 'bert.embeddings.box_layer_norm.weight', 'bert.embeddings.box_layer_norm.bias', 'bert.embeddings.tag_layer_norm.weight', 'bert.embeddings.tag_layer_norm.bias', 'bert.encoder.visn_fc.visn_fc.weight', 'bert.encoder.visn_fc.visn_fc.bias', 'bert.encoder.visn_fc.visn_layer_norm.weight', 'bert.encoder.visn_fc.visn_layer_norm.bias', 'bert.encoder.visn_fc.box_fc.weight', 'bert.encoder.visn_fc.box_fc.bias', 'bert.encoder.visn_fc.box_layer_norm.weight', 'bert.encoder.visn_fc.box_layer_norm.bias']
INFO:lxrt.modeling:Weights from pretrained model no

2
Load QA pre-trained LXMERT from snap/pretrained/BEST_EVAL_LOSS_enge_LXRT.pth 
Full loading failed!! Try partial loading!!
Successfully loaded: bert.embeddings.word_embeddings.weight
Successfully loaded: bert.embeddings.position_embeddings.weight
Successfully loaded: bert.embeddings.token_type_embeddings.weight
Successfully loaded: bert.embeddings.LayerNorm.weight
Successfully loaded: bert.embeddings.LayerNorm.bias
Successfully loaded: bert.embeddings.symbolic_embedding.weight
Successfully loaded: bert.embeddings.token_type_embeddings_visual.weight
Successfully loaded: bert.embeddings.position_embeddings_visual.weight
Successfully loaded: bert.embeddings.visn_fc.weight
Successfully loaded: bert.embeddings.visn_fc.bias
Successfully loaded: bert.embeddings.visn_layer_norm.weight
Successfully loaded: bert.embeddings.visn_layer_norm.bias
Successfully loaded: bert.embeddings.box_fc.weight
Successfully loaded: bert.embeddings.box_fc.bias
Successfully loaded: bert.embeddings.box_layer_norm.w

BertAdam Total Iters: 693350
BERTAdam lr 5e-05 total_steps 693350  warmup 0.1


In [22]:
vqa.train(vqa.train_tuple, vqa.valid_tuple)

  1%|█                                                                           | 201/13867 [00:53<1:22:42,  2.75it/s]

Epoch: 0, Iter: 200/13867
    loss    2300.633241
dtype: float64
~~~~~~~~~~~~~~~~~~



  3%|██▎                                                                           | 401/13867 [01:47<59:50,  3.75it/s]

Epoch: 0, Iter: 400/13867
    loss    2015.263159
dtype: float64
~~~~~~~~~~~~~~~~~~



  4%|███▎                                                                        | 601/13867 [02:41<1:00:27,  3.66it/s]

...

Epoch: 49, Iter: 13600/13867
    loss    0.775512
dtype: float64
~~~~~~~~~~~~~~~~~~



100%|█████████████████████████████████████████████████████████████████████████▋| 13801/13867 [1:01:21<00:16,  3.99it/s]

Epoch: 49, Iter: 13800/13867
    loss    0.7968
dtype: float64
~~~~~~~~~~~~~~~~~~



100%|████████████████████████████████████████████████████████████████████████████████| 813/813 [01:25<00:00,  9.48it/s]



Epoch 49: Train 92.59
Epoch 49: Valid 66.19
Epoch 49: Best 66.21
snap/vqa_multilingual
